In [1]:
import pandas as pd
import numpy as np
import random
import torch
TRAIN_ROWS = 10000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
IMAGE_SIZE=100

In [2]:
df = pd.read_csv("train.csv", nrows=10000)
df = df.iloc[-TRAIN_ROWS:]

In [3]:
df = df.fillna(df.mean())
df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264
...,...,...,...,...,...,...,...,...,...,...
9995,1514842740,11,3.0,338.9000,339.8000,338.9000,338.9000,20.000000,339.788006,-0.001349
9996,1514842800,2,47.0,2365.7750,2377.5500,2350.1000,2363.9450,7.480946,2363.837460,-0.004136
9997,1514842800,0,1.0,8.2889,8.2889,8.2889,8.2889,1.000000,8.288900,0.002268
9998,1514842800,1,240.0,13582.5620,13743.0000,13490.1000,13608.7080,19.605673,13597.890821,-0.001269


In [4]:
def transform_targed(df):
    new_col_name ="Target"
    df.loc[df[new_col_name] >= 0, new_col_name] = 1
    df.loc[df[new_col_name] < 0, new_col_name] = 0
    return df
    
    

In [5]:
def expand_to_ignoring_asset_it_row(df):
    Asset_ID = df["Asset_ID"]
    dm = pd.get_dummies(Asset_ID)
    columns = ["Count", "Open", "High", "Low", "Close", "Volume", "VWAP"]
    for one in range(14):
        for column in columns:
            factor = dm.get(one, 0)
            df["{}_{}".format(column, one)] = df["{}".format(column)] * factor
    del df["Asset_ID"]
    del df["timestamp"]
    for column in columns:
        del df["{}".format(column)]
    
    return df
    
    

In [6]:
def add_hyperfeatures_to_df(df):
    original_columns = list(df.columns)
    for c in original_columns:
        df["{}_ewm_mean".format(c)] = df[c].ewm(com=10).mean()
        df["{}_ewm_std".format(c)] = df[c].ewm(com=10).std()
        df["{}_ewm_cov".format(c)] = df[c].ewm(com=10).cov()

        windows = {10, 60, 120}
        shifts = [0, 120]
        for base_size in windows:
            for shift in shifts:
                df["shift_{}_{}_mean_{}".format(shift, c, base_size)] = (
                    df[c].shift(shift).rolling(window=base_size).mean()
                )
                # df["{}_var_{}".format(c, base_size)] = (
                #    df[c].rolling(window=base_size).var()
                # )
                df["shift_{}_{}_skew_{}".format(shift, c, base_size)] = (
                    df[c].shift(shift).rolling(window=base_size).skew()
                )
                df["shift_{}_{}_kurt_{}".format(shift, c, base_size)] = (
                    df[c].shift(shift).rolling(window=base_size).kurt()
                )
                df["shift_{}_{}_std_{}".format(shift, c, base_size)] = (
                    df[c].shift(shift).rolling(window=base_size).std()
                )
                # df["{}_cov_{}".format(c, base_size)] = df[c].rolling(window=base_size).cov()

    df = df.fillna(df.mean())
    return df


def standartize_df(df, scaler=None):
    from sklearn.preprocessing import StandardScaler
    scaler = scaler or StandardScaler()
    df[:] = scaler.fit_transform(df[:])
    return df, scaler
    

In [7]:
def create_dataframe(
    df, scaler=None
) -> pd.DataFrame:
    df = transform_targed(df)
    df = expand_to_ignoring_asset_it_row(df)
    del df["Target"]
    base_columns = df.columns
    
    list_of_dataframes_input = []
    list_of_dataframes_output = []
    chunk_size = 5000
    for start in range(0, df.shape[0], chunk_size):
        df_subset = df.iloc[start:start + chunk_size]
        list_of_dataframes_input.append(df_subset)
    df = None
    for df_subset in list_of_dataframes_input:
        list_of_dataframes_output.append(add_hyperfeatures_to_df(df_subset))

    df = pd.concat(list_of_dataframes_output)
    
    df = df.drop(columns=base_columns)
    
    df, scaler = standartize_df(df, scaler)
    df = df.fillna(df.mean())
    df = df.fillna(0)
    return df, scaler

In [8]:
"""
import gresearch_crypto
env = gresearch_crypto.make_env()

# Training data is in the competition dataset as usual
train_df = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', low_memory=False)
tgt_1_model.fit(train_df)
tgt_2_model.fit(train_df)
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['Target'] = tgt_1_model.predict(test_df)
    env.predict(sample_prediction_df)
"""

"\nimport gresearch_crypto\nenv = gresearch_crypto.make_env()\n\n# Training data is in the competition dataset as usual\ntrain_df = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', low_memory=False)\ntgt_1_model.fit(train_df)\ntgt_2_model.fit(train_df)\niter_test = env.iter_test()\nfor (test_df, sample_prediction_df) in iter_test:\n    sample_prediction_df['Target'] = tgt_1_model.predict(test_df)\n    env.predict(sample_prediction_df)\n"

In [9]:
import pandas as pd
import numpy as np
from pyts.image import GramianAngularField, MarkovTransitionField
from torch.utils.data import Dataset
from typing import Callable, Tuple
import torch
from torch import Tensor, nn, optim
import torchvision
import time
import copy
import cv2
from tqdm import tqdm
class InputDataset(Dataset):
    
    @staticmethod
    def transform_to_imagetensor(x):
        gadf = GramianAngularField(image_size=IMAGE_SIZE, method='difference')
        x = gadf.fit_transform(x)
        res = []
        for img in x:
            img = np.float32(img)
            color_channeled_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            color_channeled_image = np.moveaxis(color_channeled_image, 2, 0)
            res.append(color_channeled_image)
        
        return np.array(res)
    
    def __init__(
        self, df_x: pd.DataFrame, df_y: pd.DataFrame, device: torch.DeviceObjType, transform: Callable = None, train=False, real_profit=False
    ):
        self.len = len(df_x)
        assert self.len == len(df_y)
        self.device = device
        idx = np.random.permutation(df_x.index)
        df_x = df_x.reindex(idx)
        df_y = df_y.reindex(idx)
        
        self.x = df_x
        self.x = InputDataset.transform_to_imagetensor(self.x)
        self.y = df_y
        self.y = self.y.to_numpy()
        self.real_profit = real_profit

    

    def __getitem__(self, index: int) -> Tuple[Tensor, Tensor]:
        x = self.x[index]
        y = self.y[index]

        x = torch.from_numpy(x)
        y = torch.from_numpy(np.array([y]))
        x = x.to(self.device).float()
        if self.real_profit:
            y = y.to(self.device).float()
        else:
            y = y.to(self.device).long()
        return x, y[0]

    def __len__(self):
        return self.len
    
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    losses = []
    class LossRecord:
        val_loss = None
        train_loss = None
        epoch = None
    
    terminate_training = False
    for epoch in range(num_epochs):
        if terminate_training:
            break
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
            
        loss_record = LossRecord()
        loss_record.epoch = epoch
        
        loss_diff = 9999999999
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'train':
                loss_record.train_loss = epoch_loss
                
            
            if phase == 'val':
                if True:
                    best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    terminate_training=True
            

            """
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc and epoch > 0:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            """
            
            

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    import os 
    try:
        os.remove("best_model")
    except:
        pass
    torch.save(model.state_dict(), "best_model")
    
    return model, val_acc_history

In [10]:
Y = df["Target"]

In [11]:
df, scaler = create_dataframe(df)

<ipython-input-6-d9388640596b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_mean".format(c)] = df[c].ewm(com=10).mean()
<ipython-input-6-d9388640596b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_std".format(c)] = df[c].ewm(com=10).std()
<ipython-input-6-d9388640596b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

<ipython-input-6-d9388640596b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_mean".format(c)] = df[c].ewm(com=10).mean()
<ipython-input-6-d9388640596b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_std".format(c)] = df[c].ewm(com=10).std()
<ipython-input-6-d9388640596b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

<ipython-input-6-d9388640596b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["shift_{}_{}_skew_{}".format(shift, c, base_size)] = (
<ipython-input-6-d9388640596b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["shift_{}_{}_kurt_{}".format(shift, c, base_size)] = (
<ipython-input-6-d9388640596b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

<ipython-input-6-d9388640596b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["shift_{}_{}_kurt_{}".format(shift, c, base_size)] = (
<ipython-input-6-d9388640596b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["shift_{}_{}_std_{}".format(shift, c, base_size)] = (
<ipython-input-6-d9388640596b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

<ipython-input-6-d9388640596b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_mean".format(c)] = df[c].ewm(com=10).mean()
<ipython-input-6-d9388640596b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_std".format(c)] = df[c].ewm(com=10).std()
<ipython-input-6-d9388640596b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

...

<ipython-input-6-d9388640596b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_mean".format(c)] = df[c].ewm(com=10).mean()
<ipython-input-6-d9388640596b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_std".format(c)] = df[c].ewm(com=10).std()
<ipython-input-6-d9388640596b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

<ipython-input-6-d9388640596b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_mean".format(c)] = df[c].ewm(com=10).mean()
<ipython-input-6-d9388640596b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_std".format(c)] = df[c].ewm(com=10).std()
<ipython-input-6-d9388640596b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

<ipython-input-6-d9388640596b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_mean".format(c)] = df[c].ewm(com=10).mean()
<ipython-input-6-d9388640596b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_std".format(c)] = df[c].ewm(com=10).std()
<ipython-input-6-d9388640596b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

<ipython-input-6-d9388640596b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_mean".format(c)] = df[c].ewm(com=10).mean()
<ipython-input-6-d9388640596b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_std".format(c)] = df[c].ewm(com=10).std()
<ipython-input-6-d9388640596b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

<ipython-input-6-d9388640596b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_mean".format(c)] = df[c].ewm(com=10).mean()
<ipython-input-6-d9388640596b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["{}_ewm_std".format(c)] = df[c].ewm(com=10).std()
<ipython-input-6-d9388640596b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [12]:
df

,Count_0_ewm_mean,Count_0_ewm_std,Count_0_ewm_cov,shift_0_Count_0_mean_120,shift_0_Count_0_skew_120,shift_0_Count_0_kurt_120,shift_0_Count_0_std_120,shift_120_Count_0_mean_120,shift_120_Count_0_skew_120,shift_120_Count_0_kurt_120,...,shift_120_VWAP_13_kurt_10,shift_120_VWAP_13_std_10,shift_0_VWAP_13_mean_60,shift_0_VWAP_13_skew_60,shift_0_VWAP_13_kurt_60,shift_0_VWAP_13_std_60,shift_120_VWAP_13_mean_60,shift_120_VWAP_13_skew_60,shift_120_VWAP_13_kurt_60,shift_120_VWAP_13_std_60
0,-1.407100,-0.027468,-0.057878,-0.046225,-0.090063,-0.091782,-0.065065,-0.038896,-0.079696,-0.080376,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.800538,-0.483822,-0.441661,-0.046225,-0.090063,-0.091782,-0.065065,-0.038896,-0.079696,-0.080376,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.006484,-0.685551,-0.519248,-0.046225,-0.090063,-0.091782,-0.065065,-0.038896,-0.079696,-0.080376,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.408169,-0.819418,-0.562114,-0.046225,-0.090063,-0.091782,-0.065065,-0.038896,-0.079696,-0.080376,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.647728,-0.915033,-0.588521,-0.046225,-0.090063,-0.091782,-0.065065,-0.038896,-0.079696,-0.080376,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.943403,-0.927254,-0.591644,-0.101578,1.344904,1.434006,0.126710,-0.972940,-0.940571,-0.841906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,-0.985557,-0.955921,-0.598743,-0.101578,1.344904,1.434006,0.126710,-0.972940,-0.940571,-0.841906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,-0.947251,-0.983531,-0.605282,-0.133237,1.375440,1.455493,0.124973,-0.930356,-1.035036,-0.890742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,-0.989055,-1.009427,-0.611149,-0.133237,1.375440,1.455493,0.124973,-0.930356,-1.035036,-0.890742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
Y.to_numpy()

array([0., 0., 0., ..., 1., 0., 1.])

In [14]:
validation_size = int(len(df) / 5)
dataset_train = InputDataset(df_x=df.iloc[:-validation_size],df_y=Y.iloc[:-validation_size], device=DEVICE)
dataset_valid = InputDataset(df_x=df.iloc[-validation_size:],df_y=Y.iloc[-validation_size:], device=DEVICE)


train_loader1 = torch.utils.data.DataLoader(
    dataset=dataset_train, batch_size=BATCH_SIZE, drop_last=True
)
validation_loader1 = torch.utils.data.DataLoader(
    dataset=dataset_valid, batch_size=BATCH_SIZE, drop_last=True
)

dataloaders_dict = {'train':train_loader1, 'val':validation_loader1}

In [15]:
model = torchvision.models.resnet101(pretrained=True)
model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.08)



In [ ]:
model, val_acc = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=10, is_inception=False)

  0%|          | 0/125 [00:00<?, ?it/s]

Epoch 0/9
----------


/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/31 [00:00<?, ?it/s]

train Loss: 8.5998 Acc: 0.0044


  0%|          | 0/125 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

val Loss: 1.3257 Acc: 0.5015

Epoch 6/9
----------


  0%|          | 0/31 [00:00<?, ?it/s]

train Loss: 0.4857 Acc: 0.8473


 90%|█████████ | 28/31 [00:07<00:00,  3.93it/s]